<a href="https://colab.research.google.com/github/Bray-Nyagwoka/ISTG6010-2025/blob/main/Week5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import random

class ShotResult:
    HIT = "Hit"
    MISS = "Miss"
    SUNK = "Sunk"
    INVALID = "Invalid"
    ALREADY_TAKEN = "Already Taken"

class Ship:
    def __init__(self, name, size):
        self.name = name
        self.size = size
        self.positions = []
        self.hits = set()

    def place(self, start_row, start_col, horizontal):
        self.positions.clear()
        for i in range(self.size):
            row = start_row
            col = start_col
            if horizontal:
                col += i
            else:
                row += i
            self.positions.append((row, col))

    def register_hit(self, position):
        if position in self.positions:
            self.hits.add(position)
            return True
        return False

    def is_sunk(self):
        return set(self.positions) == self.hits

class Grid:
    def __init__(self, size=10):
        self.size = size
        self.ships = []
        self.shots = set()

    def is_valid_position(self, ship):
        for pos in ship.positions:
            r, c = pos
            if not (0 <= r < self.size and 0 <= c < self.size):
                return False
            for existing in self.ships:
                if pos in existing.positions:
                    return False
        return True

    def add_ship(self, ship):
        if self.is_valid_position(ship):
            self.ships.append(ship)
            return True
        return False

    def receive_shot(self, position):
        if position in self.shots:
            return ShotResult.ALREADY_TAKEN
        self.shots.add(position)
        for ship in self.ships:
            if ship.register_hit(position):
                if ship.is_sunk():
                    return ShotResult.SUNK
                return ShotResult.HIT
        return ShotResult.MISS

    def all_ships_sunk(self):
        return all(ship.is_sunk() for ship in self.ships)

    def display_own_grid(self):
        grid = [['~' for _ in range(self.size)] for _ in range(self.size)]
        for ship in self.ships:
            for r, c in ship.positions:
                grid[r][c] = 'S'
        for r, c in self.shots:
            if grid[r][c] == 'S':
                grid[r][c] = 'X'
            else:
                grid[r][c] = 'O'
        print("Brian's Grid:")
        self.print_grid(grid)

    def display_opponent_grid(self):
        grid = [['~' for _ in range(self.size)] for _ in range(self.size)]
        for r, c in self.shots:
            grid[r][c] = 'X'
        print("Opponent Grid (Brian's shots):")
        self.print_grid(grid)

    def print_grid(self, grid):
        print("  " + " ".join(str(i) for i in range(self.size)))
        for idx, row in enumerate(grid):
            print(f"{idx} " + " ".join(row))

class Player:
    def __init__(self, name):
        self.name = name
        self.grid = Grid()

    def place_ships_randomly(self, ship_definitions):
        for name, size in ship_definitions:
            placed = False
            while not placed:
                horizontal = random.choice([True, False])
                row = random.randint(0, self.grid.size - 1)
                col = random.randint(0, self.grid.size - 1)
                ship = Ship(name, size)
                ship.place(row, col, horizontal)
                placed = self.grid.add_ship(ship)

    def take_turn(self, opponent_grid):
        while True:
            try:
                user_input = input(f"{self.name}, enter shot (row,col): ")
                row, col = map(int, user_input.strip().split(','))
                if not (0 <= row < opponent_grid.size and 0 <= col < opponent_grid.size):
                    raise ValueError
                result = opponent_grid.receive_shot((row, col))
                print(f"{self.name}'s shot at ({row},{col}): {result}")
                return result
            except ValueError:
                print("Invalid input. Enter coordinates like 2,3.")

class Game:
    def __init__(self):
        self.ship_definitions = [("Destroyer", 2), ("Submarine", 3), ("Battleship", 4)]
        self.brian = Player("Brian")
        self.computer = Player("Computer")

    def setup(self):
        print("Placing Brian's ships randomly...")
        self.brian.place_ships_randomly(self.ship_definitions)
        print("Placing computer ships randomly...")
        self.computer.place_ships_randomly(self.ship_definitions)

    def play(self):
        while True:
            print("\n=== New Game Starting ===\n")
            self.setup()
            turn = 0
            while True:
                print("\n--- Current Grids ---")
                self.brian.grid.display_own_grid()
                self.computer.grid.display_opponent_grid()

                if turn % 2 == 0:
                    result = self.brian.take_turn(self.computer.grid)
                else:
                    while True:
                        row = random.randint(0, self.computer.grid.size - 1)
                        col = random.randint(0, self.computer.grid.size - 1)
                        if (row, col) not in self.brian.grid.shots:
                            break
                    result = self.brian.grid.receive_shot((row, col))
                    print(f"Computer's shot at ({row},{col}): {result}")

                if self.computer.grid.all_ships_sunk():
                    print("\n Brian wins!\n")
                    break
                elif self.brian.grid.all_ships_sunk():
                    print("\n Computer wins!\n")
                    break
                turn += 1

            play_again = input("Do you want to play again? (y/n): ").strip().lower()
            if play_again != 'y':
                print("Thanks for playing!")
                break

            self.brian = Player("Brian")
            self.computer = Player("Computer")


In [ ]:
# Run the Battleship Game with Brian
game = Game()
game.play()



=== New Game Starting ===

Placing Brian's ships randomly...
Placing computer ships randomly...

--- Current Grids ---
Brian's Grid:
  0 1 2 3 4 5 6 7 8 9
0 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
1 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
2 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
3 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
4 ~ ~ ~ S ~ ~ ~ ~ ~ ~
5 ~ ~ ~ S ~ ~ ~ ~ ~ ~
6 ~ ~ ~ S S S S S ~ ~
7 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
8 S ~ ~ ~ ~ ~ ~ ~ ~ ~
9 S ~ ~ ~ ~ ~ ~ ~ ~ ~
Opponent Grid (Brian's shots):
  0 1 2 3 4 5 6 7 8 9
0 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
1 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
2 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
3 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
4 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
5 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
6 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
7 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
8 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
9 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
